In [3]:
from requests import get
import json
from geopy.geocoders import Nominatim

# получаем название города и его координаты
city_name = input('Введите название города, погода которого вас интересует: \n')
geolocator = Nominatim(user_agent="TLV")
location = geolocator.geocode(city_name)

# это бы надо спрятать
token_yandex = '1393e7a4-cdcc-4413-9568-6a7d46ee9419'

# получаем инфу от яндекс.погоды
url = f'https://api.weather.yandex.ru/v2/informers?lat={location.latitude}&lon={location.longitude}&[lang=ru_RU]'
request = get(url, headers={'X-Yandex-API-Key': token_yandex})
data = json.loads(request.text)

#словари для парсинга инфы
conditions = {'clear': 'ясно', 'partly-cloudy': 'малооблачно', 'cloudy': 'облачно с прояснениями',
              'overcast': 'пасмурно', 'drizzle': 'морось', 'light-rain': 'небольшой дождь',
              'rain': 'дождь', 'moderate-rain': 'умеренно сильный', 'heavy-rain': 'сильный дождь',
              'continuous-heavy-rain': 'длительный сильный дождь', 'showers': 'ливень',
              'wet-snow': 'дождь со снегом', 'light-snow': 'небольшой снег', 'snow': 'снег',
              'snow-showers': 'снегопад', 'hail': 'град', 'thunderstorm': 'гроза',
              'thunderstorm-with-rain': 'дождь с грозой', 'thunderstorm-with-hail': 'гроза с градом'
              }
wind_dir = {'nw': 'северо-западное', 'n': 'северное', 'ne': 'северо-восточное', 'e': 'восточное',
            'se': 'юго-восточное', 's': 'южное', 'sw': 'юго-западное', 'w': 'западное', 'с': 'штиль'}

#парсим состояние погоды на данный момент
data['fact']['condition'] = conditions[data['fact']['condition']]
data['fact']['wind_dir'] = wind_dir[data['fact']['wind_dir']]

#парсим прогноз погоды
for parts in data['forecast']['parts']:
    parts['condition'] = conditions[parts['condition']]
    parts['wind_dir'] = wind_dir[parts['wind_dir']]

#формируем финальный результат
weather = dict()

params = ['condition', 'wind_dir', 'pressure_mm', 'humidity']

for parts in data['forecast']['parts']:
    weather[parts['part_name']] = dict()
    weather[parts['part_name']]['temp'] = parts['temp_avg']
    for param in params:
        weather[parts['part_name']][param] = parts[param]

    weather['fact'] = dict()
    weather['fact']['temp'] = data['fact']['temp']
    for param in params:
        weather['fact'][param] = data['fact'][param]

    weather['link'] = data['info']['url']

#и выводим финальный результат
day = {'night': 'ночью', 'morning': 'утром', 'day': 'днем', 'evening': 'вечером', 'fact': 'сейчас'}

print(location.address)

for i in weather.keys():
    if i != 'link':
        daytime = day[i]
        print(f'Температура {daytime} {weather[i]["temp"]} градусов, осадки: {weather[i]["condition"]}')
print(f'Более подробно: {weather["link"]}')

Калуга, городской округ Калуга, Калужская область, Центральный федеральный округ, Россия
Температура вечером 15 градусов, осадки: дождь
Температура сейчас 16 градусов, осадки: дождь
Температура ночью 14 градусов, осадки: дождь
Более подробно: https://yandex.ru/pogoda/6?lat=54.5101087&lon=36.2598115
